# <font color="#003660">Applied Machine Learning for Text Analysis (M.184.5331)</font>


# <font color="#003660">Session 6: Solving Tasks with Prompting LLMs</font>

# <font color="#003660">Notebook 2: Solving Tasks in TextWorld</font>

<center><br><img width=256 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/dag.png"/><br></center>

<p>

<div>
    <font color="#085986"><b>By the end of this lesson, you ...</b><br><br>
        ... you know what TextWorld is and how it works, <br>
        ... will learn about system prompts and meta prompting <br>
        ... will learn how to apply prompting to "automate" a process.
    </font>
</div>
</p>

The following content is heavily inspired by the following excellent sources:


* [Raschka (2024): Building a Large Language Model (From Scratch)](https://www.manning.com/books/build-a-large-language-model-from-scratch)
* [HuggingFace (2024): NLP Course](https://huggingface.co/learn/nlp-course/)
* [Huggingface (2024): Open-Source AI Cookbook](https://huggingface.co/learn/cookbook/index)
* [Prompt Engineering Guide](https://www.promptingguide.ai/)
* [Ollama](https://github.com/ollama/ollama)
* [TextWorld Documentation](https://textworld.readthedocs.io/en/stable/notes/framework.html)
* [Microsoft Research Blog](https://www.microsoft.com/en-us/research/blog/first-textworld-problems-the-competition-using-text-based-games-to-advance-capabilities-of-ai-agents/)
* [TextWorld GitHub](https://github.com/microsoft/TextWorld)
* [Côté et al. (2018)](https://doi.org/10.48550/arXiv.1806.11532)

## TextWorld and the First TextWorld Problems Challenge

TextWorld was a learning environment for text-based games, specifically designed by Microsoft to support environments for text-based reinforcement agents ([Côté et al., 2018](https://doi.org/10.48550/arXiv.1806.11532)). With the publication of TextWorld, Microsoft also published the *First TextWorld Problems Challenge* which included *The Cooking Game*, where text-based agents need to navigate through different rooms, find the recipe in the kitchen, find, cut, cook the ingredients according to the recipe and eat it, using pre-defined text-commands on a diverse set of ingredients ([Microsoft, 2019](https://www.microsoft.com/en-us/research/blog/first-textworld-problems-the-competition-using-text-based-games-to-advance-capabilities-of-ai-agents/)).


In [1]:
# Open that link to checkout TextWorld https://www.microsoft.com/en-us/research/project/textworld/try-it/

## Setup of the FTWP Challenge and its Evaluation

In [2]:
!pip install textworld termcolor

In [3]:
import os
import json
import random
from glob import glob
from typing import Mapping, Any
from termcolor import colored
from tqdm.notebook import tqdm
from IPython.display import clear_output

import numpy as np

import textworld
from textworld import EnvInfos
import textworld.gym
from textworld.gym import Agent

from ollama import Client

In [4]:
def secure_listdir(path, rm_dirs=[".ipynb_checkpoints", ]):
    files = os.listdir(path)
    for rm_dir in rm_dirs:
        if rm_dir in files:
            files.remove(rm_dir)
    return files

In [5]:
class HumanAgent(Agent):
    """ Agent that randomly selects a command from the admissible ones. """
    def __init__(self, seed=42):
        self.seed = seed
        self.messages = []
        self.messages_raw = []

    @property
    def infos_to_request(self) -> EnvInfos:
        return EnvInfos()

    def act(self, obs: str, score: int, done: bool, infos: Mapping[str, Any]) -> str:
        action = input()

        return action

In [6]:
def play(agent, path, max_step=50, seed=42, verbose=True, render=True):
    random.seed(seed)  # For reproducibility when using action sampling.
    np.random.seed(seed)  # For reproducibility when using action sampling.

    infos_to_request = agent.infos_to_request
    infos_to_request.max_score = True  # Needed to normalize the scores.

    gamefiles = [path]
    if os.path.isdir(path):
        gamefiles = glob(os.path.join(path, "*.z8"))

    env_id = textworld.gym.register_games(
        gamefiles,
        request_infos=infos_to_request,
        max_episode_steps=max_step
    )
    
    env = textworld.gym.make(env_id)  # Create a Gym environment to play the text game.
    if verbose:
        if os.path.isdir(path):
            print(os.path.dirname(path))
        else:
            print(os.path.basename(path))
    
    obs, infos = env.reset()  # Start new episode.
    if render:
        env.render()  # Print the initial observation.

    obs_template_violation = "Your answer was not conform to the template provided. Provide the answer according to your taks description."

    score = 0
    done = False
    moves = 0
    while not done:
        command = agent.act(obs, score, done, infos)
        
        rethink_counter = 0
        while len(command) > 50:
            command = agent.act(obs_template_violation, score, done, infos)
            rethink_counter += 1
            moves += 1
            if rethink_counter == 3:
                break
        if len(command) > 50:
            command = "I don't know"
        
        obs, score, done, infos = env.step(command)
        if render:
            env.render()  # Print the command's feedback.
        moves += 1

    agent.act(obs, score, done, infos)  # Let the agent know the game is done.

    norm_score = score / infos["max_score"]

    env.close()

    if verbose:
        msg = "Moves: {:3d}\tScore: {:4d} / {:4d}\tNormalized Score: {:3.2f}"
        print(msg.format(moves, score, infos["max_score"], norm_score))
    
    return moves, score, norm_score, infos["max_score"], agent.messages, agent.messages_raw

In [7]:
class LLMAgent(Agent):
    """ LLM agent. """
    def __init__(
            self, 
            llm, 
            prompt,
            history_length, 
            system_message,
            examples, 
            env_infos={}, 
            seed=42,
            render=False,
            options={"seed": 42},
        ):
        self.llm = llm
        self.prompt = prompt
        self.history_length = history_length
        self.messages = [
            {
                'role': 'system', 
                'content': system_message, 
            }, 
        ]
        self.examples = examples
        self.messages += self.examples
        self.env_infos = EnvInfos(**env_infos)
        self.seed = seed
        self.render = render
        self.recipe = "You have not read the recipe yet."
        self.messages_raw = []
        self.options=options

    @property
    def infos_to_request(self) -> EnvInfos:
        return self.env_infos

    def act(self, obs: str, score: int, done: bool, infos: Mapping[str, Any]) -> str:
        inventory = infos.get("inventory", "").strip()
        description = infos.get("description", "").strip()
        feedback = obs.strip()

        if 'You open the copy of "Cooking: A Modern Approach (3rd Ed.)" and start reading:' in feedback:
            self.recipe = feedback.replace('You open the copy of "Cooking: A Modern Approach (3rd Ed.)" and start reading:', "").strip()

        prompt = self.prompt.format(feedback=feedback, recipe=self.recipe, inventory=inventory, description=description)
        self.messages_raw.append({'role': 'user', "feedback": feedback, "recipe": self.recipe, "inventory": inventory, "description": description})
        self.messages.append({'role': 'user', 'content': prompt})
        if self.render:
            print(prompt)
        
        context_messages = self.messages[0:len(self.examples) + 1] + self.messages[1:][-min(self.history_length * 2 + 1, len(self.messages) - (len(self.examples) + 1)):]

        answer = client.chat(model=self.llm, messages=context_messages, options=self.options).get("message", {}).get("content", "")
        self.messages.append({'role': 'assistant', 'content': answer})
        self.messages_raw.append({'role': 'assistant', 'content': answer})
        if self.render:
            print(colored("> " + answer, "yellow"))
        answer = answer.split(":")[-1].strip()
        
        return answer

In [8]:
def evaluate_agent(MODEL_NAME, PROMPT, HISTORY_LENGTH, SYSTEM_MESSAGE, EXAMPLES, ENV_INFOS, train_files, train_path, RENDER_AGENT, RENDER, RESULTS_PATH, OPTIONS):
    score_sum = 0
    max_score_sum = 0
    wins = 0
    train_results = {}
    for train_file in tqdm(train_files):
        train_file_path = os.path.join(train_path, train_file)
        config_file_path = train_file_path.replace(".z8", ".json")
        config = {}
        with open(config_file_path) as config_file:
            config = json.load(config_file)
        
        walkthrough = config.get("extras", {}).get("walkthrough", [])
        min_moves = len(walkthrough)
        recipe = config.get("extras", {}).get("recipe", "")
        
        moves, score, norm_score, max_score, messages, messages_raw = play(LLMAgent(MODEL_NAME, PROMPT, HISTORY_LENGTH, SYSTEM_MESSAGE, EXAMPLES, ENV_INFOS, 42, render=RENDER_AGENT, options=OPTIONS), train_file_path, render=RENDER)
        train_results[train_file] = {"moves": moves, "min_moves": min_moves, "score": score, "norm_score": norm_score, "max_score": max_score, "recipe": recipe, "walkthrough":walkthrough, "messages": messages, "messages_raw": messages_raw}
        with open(RESULTS_PATH, "w") as f:
            json.dump(train_results, f)
        score_sum += score
        max_score_sum += max_score
        if score == max_score:
            wins += 1
    print(f"Average normalized score: {score_sum / max_score_sum}")
    print(f"Win rate: {wins / len(train_files)}")
    print(f"Overall wins: {wins}")

We will work with a small subset of the FTWP challenge, to ensure the feasibility of the game with prompting.

The agent:
* starts in the kitchen
* needs to:
    - read the recipe
    - find all ingredients in the kitchen (maybe open the fridge)
    - take all ingredients and the knife
    - cut all ingredients according to the recipe
    - cook all ingredients according to the recipe
    - prepare the meal
    - eat the meal
* gets the following information in every round:
    - the game engines feedback
    - the recipe (if it has read it)
    - the inventory
    - the description of the current room

Let's try this out:

In [10]:
play(HumanAgent(), "games/train/tw-cooking-recipe1+cut-OYmoIYkTvbNcxja.z8", render=True)

tw-cooking-recipe1+cut-OYmoIYkTvbNcxja.z8



                    ________  ________  __    __  ________
                   |        \|        \|  \  |  \|        \
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$
                      | $$   | $$__     \$$\/  $$   | $$
                      | $$   | $$  \     >$$  $$    | $$
                      | $$   | $$$$$    /  $$$$\    | $$
                      | $$   | $$_____ |  $$ \$$\   | $$
                      | $$   | $$     \| $$  | $$   | $$
                       \$$    \$$$$$$$$ \$$   \$$    \$$
              __       __   ______   _______   __        _______
             |  \  _  |  \ /      \ |       \ |  \      |       \
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             | $$ $$\$$\$$| $$  | $$| $$$$$$$\| $$      | $$  | $$
             | $$$$  \$$$$| $$__/ $

(6, 3, 1.0, 3, [], [])

Now let's finish the environmental setup and start prompting:

In [9]:
RANDOM_SEED = 42
OPTIONS = {
    "seed": RANDOM_SEED,
    "num_ctx": 8192,
    "num_predict": -2,
}

proteus_host='http://131.234.154.103:11434'
client = Client(host=proteus_host)

games_dir = "games"

train_split = "train"
train_path = os.path.join(games_dir, train_split)
train_files = secure_listdir(train_path)
train_files = [x for x in train_files if x.endswith(".z8")]

valid_split = "valid"
valid_path = os.path.join(games_dir, valid_split)
valid_files = secure_listdir(valid_path)
valid_files = [x for x in valid_files if x.endswith(".z8")]

test_split = "test"
test_path = os.path.join(games_dir, test_split)
test_files = secure_listdir(test_path)
test_files = [x for x in test_files if x.endswith(".z8")]

HISTORY_LENGTH = 3
RENDER_AGENT = True # renders processed the agent's view
RENDER = False # renders the unprocessed game view
MODEL_NAME = "gemma2" # 9 billion parameter model

ENV_INFOS = {
    "description": True,
    "inventory": True
}

## System Prompts and Meta Prompting

*System prompt* is a common phrase describing general basic instruction and a general behavior the LLM conversational agent should adhere to ([Li et al., 2024](https://doi.org/10.48550/arXiv.2402.10962)). 

A *meta prompt*, in contrast, is an example-agnostic structured prompt that captures the reasoning structure of a specific task category ([Zhang et al., 2024](https://doi.org/10.48550/arXiv.2311.11482))

While these two have distinct definition, [Zhang et al. (2024)](https://doi.org/10.48550/arXiv.2311.11482) define also structured system meta prompts:

![System and Meta Prompt](imgs/sys_meta.png)

([Zhang et al., 2024](https://doi.org/10.48550/arXiv.2311.11482))

Let's adapt this to our cooking task in TextWorld.

##### Training (Prompt Engineering)

In [10]:
EXAMPLES = { # you can enter example messages for one-shot or few-shot prompting here

}

In [11]:
SYSTEM_MESSAGE = """You are the player of a text-based cooking game.

These are the important game entities: 
cookbook:       Cookbook that contains the recipe.
<ingredient>:   Ingredient you can take, drop, cut, or cook according to the recipe.
<container>:    Container you can open, which can contain ingredients.
<door>:         Door you can open.
knife:          Knife to slice, dice, chop ingredients.
stove:          Stove to fry ingredients.
oven:           Oven to roast ingredients.
bbq:            BBQ to grill ingredients.
meal:           Meal you can eat.

These are the commands to interact with the game:
look
open <container>
close <container>
open <door>
close <door>
inventory
take <ingredient>
take knife
read cookbook
slice <ingredient> with knife
dice <ingredient> with knife
chop <ingredient> with knife
cook <ingredient> with stove    (fry an ingredient)
cook <ingredient> with oven     (roast an ingredient)
cook <ingredient> with bbq      (grill an ingredient)
prepare meal
eat meal

This is the basic game process:
Navigate to the kitchen and read the cookbook.
Take all ingredients according to the recipe.
Follow the recipe.
Eat the meal.

Your main task:
You provide answers to game engine queries following the command templates and game process above.
Never provide answers deviating from the command templates above - unless explicitly asked by the game engine.
Provide only one command at a time - unless the game engine explicitly asks to do something else."""

PROMPT = """# Current game state:
## {feedback}

## {recipe}

## {inventory}

## You are in the {description}"""


STRATEGY_NAME = "BASIC"
RESULTS_PATH = os.path.join("results", train_split, f"{STRATEGY_NAME}.json")

evaluate_agent(MODEL_NAME, PROMPT, HISTORY_LENGTH, SYSTEM_MESSAGE, EXAMPLES, ENV_INFOS, train_files[0:1], train_path, RENDER_AGENT, RENDER, RESULTS_PATH, OPTIONS) # evaluate on a single game to show the results

  0%|          | 0/1 [00:00<?, ?it/s]

tw-cooking-recipe3+cut-paplho1xUDnvIRKO.z8
# Current game state:
## ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
                      | $$   | $$     \| $$  | $$   | $$          
                       \$$    \$$$$$$$$ \$$   \$$    \$$          
              __       __   ______   _______   __        _______  
             |  \  _  |  \ /      \ |       \ |  \      |       \ 
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             |

In [14]:
evaluate_agent(MODEL_NAME, PROMPT, HISTORY_LENGTH, SYSTEM_MESSAGE, EXAMPLES, ENV_INFOS, train_files, train_path, False, RENDER, RESULTS_PATH, OPTIONS) # skip rendering during evaluation

  0%|          | 0/20 [00:00<?, ?it/s]

tw-cooking-recipe3+cut-paplho1xUDnvIRKO.z8
Moves:  18	Score:    2 /    4	Normalized Score: 0.50
tw-cooking-recipe3+take3+cook-nBkmUV0RsaemSLkZ.z8
Moves:  11	Score:    7 /    7	Normalized Score: 1.00
tw-cooking-recipe1+take1+cook+cut+open-EOGWTmvqhKZaiBgm.z8
Moves:  15	Score:    5 /    5	Normalized Score: 1.00
tw-cooking-recipe2+take2+cook-NeVNUnMLUynRu8J8.z8
Moves:  50	Score:    2 /    5	Normalized Score: 0.40
tw-cooking-recipe1+take1+cut-JLKeIN6Ysvb2uD6Q.z8
Moves:   6	Score:    2 /    4	Normalized Score: 0.50
tw-cooking-recipe2+take2+cook-9WLOIdeFGqliDKG.z8
Moves:   5	Score:    4 /    4	Normalized Score: 1.00
tw-cooking-recipe2+take2+cook-xWE6IypEsn7QHldV.z8
Moves:   6	Score:    5 /    5	Normalized Score: 1.00
tw-cooking-recipe1+cut-OYmoIYkTvbNcxja.z8
Moves:   6	Score:    1 /    3	Normalized Score: 0.33
tw-cooking-recipe2+take2+cook+cut-KNBVfrypS2BbsEl.z8
Moves:  13	Score:    8 /    8	Normalized Score: 1.00
tw-cooking-recipe2+take2+cut+open-16bRf1bOfGG3uodv.z8
Moves:  20	Score:    4 /

We can add CoT prompting to this and check out, if we can improve the preformance.

Additionally this concept is called *Instruction-style* prompting ([Zhou et al., 2023](https://doi.org/10.18653/v1/2023.findings-emnlp.968)). 

In [14]:
# The first line is part of the prompt is the instruction and chain-of-thought
PROMPT = """Reason step-by-step about the feedback, the recipe, the items you are carrying and the room you are in and provide the next command. Use the format REASONING: <your reasoning here> NEXT COMMAND: <your next command here>
# Current game state:
## {feedback}

## {recipe}

## {inventory}

## You are in the {description}"""


STRATEGY_NAME = "CoT"
RESULTS_PATH = os.path.join("results", train_split, f"{STRATEGY_NAME}.json")

evaluate_agent(MODEL_NAME, PROMPT, HISTORY_LENGTH, SYSTEM_MESSAGE, EXAMPLES, ENV_INFOS, train_files, train_path, False, RENDER, RESULTS_PATH, OPTIONS) # skip rendering during evaluation

  0%|          | 0/20 [00:00<?, ?it/s]

tw-cooking-recipe3+cut-paplho1xUDnvIRKO.z8
Moves:   6	Score:    1 /    4	Normalized Score: 0.25
tw-cooking-recipe3+take3+cook-nBkmUV0RsaemSLkZ.z8
Moves:   8	Score:    7 /    7	Normalized Score: 1.00
tw-cooking-recipe1+take1+cook+cut+open-EOGWTmvqhKZaiBgm.z8
Moves:  12	Score:    5 /    5	Normalized Score: 1.00
tw-cooking-recipe2+take2+cook-NeVNUnMLUynRu8J8.z8
Moves:   8	Score:    5 /    5	Normalized Score: 1.00
tw-cooking-recipe1+take1+cut-JLKeIN6Ysvb2uD6Q.z8
Moves:   6	Score:    2 /    4	Normalized Score: 0.50
tw-cooking-recipe2+take2+cook-9WLOIdeFGqliDKG.z8
Moves:   5	Score:    4 /    4	Normalized Score: 1.00
tw-cooking-recipe2+take2+cook-xWE6IypEsn7QHldV.z8
Moves:   7	Score:    5 /    5	Normalized Score: 1.00
tw-cooking-recipe1+cut-OYmoIYkTvbNcxja.z8
Moves:   6	Score:    1 /    3	Normalized Score: 0.33
tw-cooking-recipe2+take2+cook+cut-KNBVfrypS2BbsEl.z8
Moves:  11	Score:    8 /    8	Normalized Score: 1.00
tw-cooking-recipe2+take2+cut+open-16bRf1bOfGG3uodv.z8
Moves:  10	Score:    4 /

Improved only by 3 wins. Let's check if this improvement is "real" on our validation dataset. (The validation dataset is not our test dataset.)

##### Validation (distinct from Test)

In [12]:
PROMPT = """# Current game state:
## {feedback}

## {recipe}

## {inventory}

## You are in the {description}"""


STRATEGY_NAME = "BASIC"
RESULTS_PATH = os.path.join("results", valid_split, f"{STRATEGY_NAME}.json")

evaluate_agent(MODEL_NAME, PROMPT, HISTORY_LENGTH, SYSTEM_MESSAGE, EXAMPLES, ENV_INFOS, valid_files, valid_path, False, RENDER, RESULTS_PATH, OPTIONS) # skip rendering during evaluation

  0%|          | 0/15 [00:00<?, ?it/s]

tw-cooking-recipe3+take3-5MaRuLGRCgl3F6Bm.z8
Moves:  50	Score:    3 /    5	Normalized Score: 0.60
tw-cooking-recipe3+take3+cook-xQ0of98VFPlOfpvD.z8
Moves:   9	Score:    7 /    7	Normalized Score: 1.00
tw-cooking-recipe1+take1+cut-W5NyHaWWHY2nh3Vj.z8
Moves:  10	Score:    4 /    4	Normalized Score: 1.00
tw-cooking-recipe1+take1+cook+cut-3Zxahr7mTnGQc5QM.z8
Moves:   8	Score:    5 /    5	Normalized Score: 1.00
tw-cooking-recipe2+cut-pY7eFyRjs3lPhZb3.z8
Moves:  50	Score:    0 /    2	Normalized Score: 0.00
tw-cooking-recipe3+cut-JRBqI09dTv7liZmN.z8
Moves:   8	Score:    5 /    5	Normalized Score: 1.00
tw-cooking-recipe2+take2+cook+cut-me7xiaQyI3JQiZ3j.z8
Moves:   9	Score:    6 /    6	Normalized Score: 1.00
tw-cooking-recipe3+cook-gLK1H6peHZKpTqgj.z8
Moves:  11	Score:    4 /    4	Normalized Score: 1.00
tw-cooking-recipe1+take1+open-75glH2JbfWY0tyK2.z8
Moves:  23	Score:    1 /    3	Normalized Score: 0.33
tw-cooking-recipe3+take3+cut-jJe7Ty5DFyaYtN9E.z8
Moves:   5	Score:    3 /    5	Normalized S

In [13]:
# The first line is part of the prompt is the instruction and chain-of-thought
PROMPT = """Reason step-by-step about the feedback, the recipe, the items you are carrying and the room you are in and provide the next command. Use the format REASONING: <your reasoning here> NEXT COMMAND: <your next command here>
# Current game state:
## {feedback}

## {recipe}

## {inventory}

## You are in the {description}"""

STRATEGY_NAME = "CoT"
RESULTS_PATH = os.path.join("results", valid_split, f"{STRATEGY_NAME}.json")

evaluate_agent(MODEL_NAME, PROMPT, HISTORY_LENGTH, SYSTEM_MESSAGE, EXAMPLES, ENV_INFOS, valid_files, valid_path, False, RENDER, RESULTS_PATH, OPTIONS) # skip rendering during evaluation

  0%|          | 0/15 [00:00<?, ?it/s]

tw-cooking-recipe3+take3-5MaRuLGRCgl3F6Bm.z8
Moves:   6	Score:    3 /    5	Normalized Score: 0.60
tw-cooking-recipe3+take3+cook-xQ0of98VFPlOfpvD.z8
Moves:   9	Score:    7 /    7	Normalized Score: 1.00
tw-cooking-recipe1+take1+cut-W5NyHaWWHY2nh3Vj.z8
Moves:   6	Score:    2 /    4	Normalized Score: 0.50
tw-cooking-recipe1+take1+cook+cut-3Zxahr7mTnGQc5QM.z8
Moves:   8	Score:    5 /    5	Normalized Score: 1.00
tw-cooking-recipe2+cut-pY7eFyRjs3lPhZb3.z8
Moves:   5	Score:    0 /    2	Normalized Score: 0.00
tw-cooking-recipe3+cut-JRBqI09dTv7liZmN.z8
Moves:   8	Score:    5 /    5	Normalized Score: 1.00
tw-cooking-recipe2+take2+cook+cut-me7xiaQyI3JQiZ3j.z8
Moves:   9	Score:    6 /    6	Normalized Score: 1.00
tw-cooking-recipe3+cook-gLK1H6peHZKpTqgj.z8
Moves:   8	Score:    4 /    4	Normalized Score: 1.00
tw-cooking-recipe1+take1+open-75glH2JbfWY0tyK2.z8
Moves:   6	Score:    3 /    3	Normalized Score: 1.00
tw-cooking-recipe3+take3+cut-jJe7Ty5DFyaYtN9E.z8
Moves:   5	Score:    3 /    5	Normalized S

### Test (This would be done after defining and improving diverse prompting approaches)

In [17]:
PROMPT = """# Current game state:
## {feedback}

## {recipe}

## {inventory}

## You are in the {description}"""


STRATEGY_NAME = "BASIC"
RESULTS_PATH = os.path.join("results", test_split, f"{STRATEGY_NAME}.json")

evaluate_agent(MODEL_NAME, PROMPT, HISTORY_LENGTH, SYSTEM_MESSAGE, EXAMPLES, ENV_INFOS, test_files, test_path, False, RENDER, RESULTS_PATH, OPTIONS)

  0%|          | 0/17 [00:00<?, ?it/s]

tw-cooking-recipe4+cook-jm23SELjcxL8SQeQ.z8
Moves:  12	Score:    1 /    4	Normalized Score: 0.25
tw-cooking-recipe1+take1+cook-DBxpUJMPFb8eUdro.z8
Moves:   7	Score:    4 /    4	Normalized Score: 1.00
tw-cooking-recipe5+cut-2j6BTyrjfgg7tkM9.z8
Moves:  20	Score:    1 /    6	Normalized Score: 0.17
tw-cooking-recipe1+take1+cut+open-2EdDcloyF9XacgEK.z8
Moves:  32	Score:    1 /    3	Normalized Score: 0.33
tw-cooking-recipe3+take3+cook+cut+open-O6njCNDlsbEpiQ1X.z8
Moves:  28	Score:    9 /   11	Normalized Score: 0.82
tw-cooking-recipe4+cut-lJy7tKLOi8potxRB.z8
Moves:  10	Score:    1 /    5	Normalized Score: 0.20
tw-cooking-recipe5+cook+cut-9xrptEgBcMYDFmDd.z8
Moves:  50	Score:    0 /    6	Normalized Score: 0.00
tw-cooking-recipe3+take3+cook+open-WqaXuaJgCXKYHv6.z8
Moves:  33	Score:    8 /    8	Normalized Score: 1.00
tw-cooking-recipe2+take2+cook+cut+open-VgGqFpLpCvMEiykm.z8
Moves:  10	Score:    7 /    7	Normalized Score: 1.00
tw-cooking-recipe4+take4+cook+cut-q8RmFgxxTQQlSoEe.z8
Moves:  12	Scor

In [16]:
# The first line is part of the prompt is the instruction and chain-of-thought
PROMPT = """# Instruction: Reason step-by-step about the game engines feedback, the recipe, your inventory, and your location.
# Current game state:
## {feedback}

## {recipe}

## {inventory}

## You are in the {description}

# Question: What is the next command based on your reasoning?
Use the format 'REASONING: <reasoning> NEXT COMMAND: <command>' for your answer."""


STRATEGY_NAME = "CoT"
RESULTS_PATH = os.path.join("results", test_split, f"{STRATEGY_NAME}.json")

evaluate_agent(MODEL_NAME, PROMPT, HISTORY_LENGTH, SYSTEM_MESSAGE, EXAMPLES, ENV_INFOS, test_files, test_path, False, RENDER, RESULTS_PATH, OPTIONS)

  0%|          | 0/17 [00:00<?, ?it/s]

tw-cooking-recipe4+cook-jm23SELjcxL8SQeQ.z8
Moves:   4	Score:    0 /    4	Normalized Score: 0.00
tw-cooking-recipe1+take1+cook-DBxpUJMPFb8eUdro.z8
Moves:   6	Score:    4 /    4	Normalized Score: 1.00
tw-cooking-recipe5+cut-2j6BTyrjfgg7tkM9.z8
Moves:  50	Score:    0 /    6	Normalized Score: 0.00
tw-cooking-recipe1+take1+cut+open-2EdDcloyF9XacgEK.z8
Moves:   7	Score:    3 /    3	Normalized Score: 1.00
tw-cooking-recipe3+take3+cook+cut+open-O6njCNDlsbEpiQ1X.z8
Moves:  20	Score:   11 /   11	Normalized Score: 1.00
tw-cooking-recipe4+cut-lJy7tKLOi8potxRB.z8
Moves:   6	Score:    0 /    5	Normalized Score: 0.00
tw-cooking-recipe5+cook+cut-9xrptEgBcMYDFmDd.z8
Moves:  15	Score:    4 /    6	Normalized Score: 0.67
tw-cooking-recipe3+take3+cook+open-WqaXuaJgCXKYHv6.z8
Moves:  12	Score:    8 /    8	Normalized Score: 1.00
tw-cooking-recipe2+take2+cook+cut+open-VgGqFpLpCvMEiykm.z8
Moves:   5	Score:    2 /    7	Normalized Score: 0.29
tw-cooking-recipe4+take4+cook+cut-q8RmFgxxTQQlSoEe.z8
Moves:  12	Scor

What you have learned so far:
* *System prompts* can be used to align a model generally on a task.
* *Meta prompts* are distinct from system prompts and are basically providing a structure for reasoning in LLMs.
* However, meta prompts can also be used in system prompting.
* Multiple prompting methods (such as meta and CoT prompting) can be combined to improve results.

## Your Turn!

Now you can try to define and change prompts by yourself.

If you need some inspiration, refer to:
* [Prompt Engineering Guide](https://www.promptingguide.ai/)
* [The Prompt Report](https://doi.org/10.48550/arXiv.2406.06608)
* [OpenWebUI Prompts](https://openwebui.com/prompts/)

##### Trainig (Prompt Engineering)

In [ ]:
SYSTEM_MESSAGE = """<your magic here>"""

EXAMPLES = {

}

PROMPT = """<your magic here>"""


STRATEGY_NAME = "my_magic"
RESULTS_PATH = os.path.join("results", train_split, f"{STRATEGY_NAME}.json")

evaluate_agent(MODEL_NAME, PROMPT, HISTORY_LENGTH, SYSTEM_MESSAGE, EXAMPLES, ENV_INFOS, train_files, train_path, False, RENDER, RESULTS_PATH, OPTIONS)

##### Validation

In [ ]:
SYSTEM_MESSAGE = """<your magic here>"""

EXAMPLES = {

}

PROMPT = """<your magic here>"""


STRATEGY_NAME = "my_magic"
RESULTS_PATH = os.path.join("results", valid_split, f"{STRATEGY_NAME}.json")

evaluate_agent(MODEL_NAME, PROMPT, HISTORY_LENGTH, SYSTEM_MESSAGE, EXAMPLES, ENV_INFOS, valid_files, valid_split, False, RENDER, RESULTS_PATH, OPTIONS)

##### Already 20 Wins in the Valid Set? Try to evaluate with the test set:

In [ ]:
SYSTEM_MESSAGE = """<your magic here>"""

EXAMPLES = {

}

PROMPT = """<your magic here>"""


STRATEGY_NAME = "my_magic"
RESULTS_PATH = os.path.join("results", test_split, f"{STRATEGY_NAME}.json")

evaluate_agent(MODEL_NAME, PROMPT, HISTORY_LENGTH, SYSTEM_MESSAGE, EXAMPLES, ENV_INFOS, test_files, test_split, RENDER_AGENT, RENDER, RESULTS_PATH, OPTIONS)

## Key Findings:
* Prompting and prompt engineering are more complex than it seems
* Use a train set to develop prompts, validations set to improve prompts and a test set to evaluate prompts.
* Few-shot prompting can improve but also bias your results. Therefore, meta prompting can be useful in complex environments and conversations.
* Without pre-defined techniques and systematically, you know that it works, but you don't know why! Apply systematic analysis and track changes while prompt engineering.
* Sometimes even complex prompting techniques do not improve results --> refer to fine-tuning in the next session